<a href="https://colab.research.google.com/github/Kinds-of-Intelligence-CFI/measurement-layout-tutorial/blob/main/tutorial-notebooks/4_BuildingComplexMeasurementLayouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Complex Measurement Layouts for Cognitive Benchmarks

**Lead Presenter**: Kozzy Voudouris

In this tutorial, we attempt to build a more complex measurement layout for some real agents. We incrementally build a measurement layout for studying the cognitive capability of object permanence in a complex three-dimensional environment. Finally, We evaluate some agents on a suite of tests.

## Preamble

First, let's import the libraries and functions that we will need.

In [ ]:
!pip install arviz --quiet
!pip install erroranalysis --quiet
!pip install numpy --quiet
!pip install pymc --quiet

In [ ]:
import arviz as az
import erroranalysis as ea
import gc
import graphviz
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import random as rm
import seaborn as sns

from IPython.display import Image
from scipy import stats
from sklearn.model_selection import train_test_split
from google.colab import files
from pymc import model
from xgboost import XGBRegressor
from xgboost import XGBClassifier

rng = np.random.default_rng(1997) # set a seed for reproducibility

print(f"Running on PyMC v{pm.__version__}") #Note, colab imports an older version of PyMC by default. This won't cause problems for this tutorial, but may do if you use a different backend (e.g., gpu) and a jax/numpyro sampler. In which case, run `!pip install 'pymc>5.9' --quiet`

## Data

Now let's import and inspect the data. We have data to explore from RL agents as well as humans.

In [ ]:
agent_data_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/measurement-layout-tutorial/main/data/4_objectPermanenceDataAgents.csv'
opiaagets_dataset = pd.read_csv(agent_data_url)

human_data_url = 'https://raw.githubusercontent.com/Kinds-of-Intelligence-CFI/measurement-layout-tutorial/main/data/4_objectPermanenceDataHuman.csv'
human_dataset = pd.read_csv(human_data_url)

Let's inspect the datasets. There are **4202 instances** and **15 metafeatures** for the performances of **4 agents**. There are **1608 instances** and **15 metafeatures** for human performances.

The metafeatures are as follows:
1. `basicTask` - is the task a basic task? Values (discrete, binary): `0` (No), `1` (Yes).
2. `pctbGridTask` - is the task a PCTB Grid task? Values (discrete binary): `0` (No), `1` (Yes).
3. `pctb3CupTask` - is the task a PCTB Cup task? Values (discrete binary): `0` (No), `1` (Yes).
3. `cvchickTask` - is the task a CV Chick task? Values (discrete binary): `0` (No), `1` (Yes).
3. `mainGoalSize` - what size is the goal in the task? Value range: `0.5` - `5.0`.
4. `lavaPresence` - does the instance contain lava? Values (discrete binary): `0` (No), `1` (Yes).
5. `goalLeft` - is there a goal placed to the left? Values (discrete binary): `0` (No), `1` (Yes).
5. `goalRight` - is there a goal placed to the right? Values (discrete binary): `0` (No), `1` (Yes).
5. `goalCentre` - is there a goal placed to ahead? Values (discrete binary): `0` (No), `1` (Yes).
5. `goalOccluded` - is the goal occluded when the agent starts the episode? Values (discrete binary): `0` (No), `1` (Yes).
6. `navigationDistanceGoal` - how far is the goal from the agent? This is calculated is the manhattan distance to the goal, avoiding any obstacles/pits. Value range: `9.0` - `118.0`.
6. `euclideanDistanceGoal` - how far is the goal from the agent? This is calculated is the euclidean distance to the goal, not avoiding any obstacles. Value range: `9.0` - `38.40`.
7. `navigationTurnsGoal` - how many right-angle turns would the agent take on the trajectory described by `navigationDistanceGoal`. Value range: `0.0` - `13.0`.
7. `navigationDistanceChoice` - how far is the choice point from the agent? This is calculated is the manhattan distance to the choice point, avoiding any obstacles/pits. Value range: `1.0` - `53.5`.
7. `navigationTurnsChoice` - how many right-angle turns would the agent take on the trajectory described by `navigationDistanceGoal`. Value range: `0.0` - `6.0`.
8. `numChoices` - how many choices does the agent have in the task? Value range: `1.0` - `12.0`.

The agents performed each of these tasks, and whether they obtained the goal (`1`) or not (`0`) and whether they picked the correct choice (`1`) or not (`0`) was recorded. The agents are as follows:
1. `Random_Agent` - An agent which randomly samples one of the 9 actions in the Animal-AI Environment (`no action`, `forwards`, `backwards`, `left rotate`, `right rotate`, `forwards left`, `forwards right`, `backwards left`, `backwards right`). It then takes that action for a number of steps sampled from $U(1, 20)$.
2. `Heuristic_Agent` - An agent that navigates towards green goals, following a rigid rule.
4. `Dreamer` - A dreamer-v3 agent trained for 10M steps on the set of 2372 basic and practice tasks.
6. `PPO` - A PPO agent trained for 10M steps on the set of 2372 basic and practice tasks.

In [ ]:
opiaagets_dataset

In [ ]:
human_dataset

# Building A Complex Measurement Layout

We will incrementally build a complex measurement layout for studying object permanence. We will evaluate this measurement layout by using the two reference agents, `Random_Agent` and `Heuristic_Agent`, as baselines. Finally, we will apply this measurement layout to the dreamer and PPO agents.

## Object Permanence

First, let's set up a simple measurement layout for the *Object Permanence* ability.

The leaf node of the measurement layout is success in this case, so we want to define it as a Bernoulli, taking a probability of success. Therefore, we'll need a logistic function. Because we are dealing with bounded capabilities, the logistic function means that the probability of success on a task with a minimum demand for an agent with maximum ability is 0.999. Alternatively, for an agent with minimum ability performing on a task with maximum demand, we get a probability of success of 0.001. This is a nice parameterisation of the logistic function for our case of bounded capabilities.

In [ ]:
def logistic999(x, min, max):    # This logistic function ensures that if x is at -(max-min), we get prob 0.001, and if x is at (max-min), we get prob 0.999
  x = x - min
  max = max - min
  x = 6.90675478 * x / max
  return 1 / (1 + np.exp(-x))

Try out the logistic here. Let's say you have an ability bounded between 0 and 100. An agent with maximum ability (100) performing on a task with the minimum demand (0) should pass with 0.999 probability. An agent with minimum ability (0) performing on a task with the maximum demand should pass with 0.001 probability. That means the minimum margin is $-(100-0)$ and the maximum margin is $(100-0)$.

In [ ]:
xpoints = np.linspace(-100, 100, 1000)
ypoints = logistic999(xpoints, 0, 100)
plt.plot(xpoints, ypoints)
plt.show()

Let's build an initial measurement layout. Let's define the combined object permanence demand as the distance to the goal (`navigationDistanceGoal`), which serves as a proxy for time under occlusion, multiplied by the number of choice (`numChoices`) multiplied by whether or not a goal is present (`goalOccluded`).

In [ ]:
def setupOPModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility =

  maxPermAbility =

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility


  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility =                     #which prior is appropriate?

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistance", data["navigationDistanceGoal"].values) #this is a proxy for how long the goal is occluded for.
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)

    # Margins

    objPermMargin =            # define the margin as the ability minus the combined
    objPermP =                 # apply the logistic999 transformation with minPermAbility and maxPermAbility

    taskSuccess = pm.Bernoulli("Success", objPermP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
# @title A Recommended Solution
def setupOPModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).min())

  maxPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).max())

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility


  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistance", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)

    # Margins

    objPermMargin = (objPermAbility - ((goalDist * opTest) * numChoices))
    objPermP = pm.Deterministic("objPermP", logistic999(objPermMargin, min = minPermAbility, max = maxPermAbility))

    taskSuccess = pm.Bernoulli("Success", objPermP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = setupOPModel(opiaagets_dataset, 'Random_Agent_Success')
gv = pm.model_graph.model_to_graphviz(m)
gv

Let's run this measurement layout on two of our agents: the `Random_Agent` agent and the `Heuristic_Agent`.

In [ ]:
model_random, abilityMin, abilityMax = setupOPModel(opiaagets_dataset, 'Random_Agent_Success')
with model_random:
  data_random = pm.sample(1000, target_accept=0.95, random_seed=rng)

model_heuristic, abilityMin, abilityMax = setupOPModel(opiaagets_dataset, 'Heuristic_Agent_Success')
with model_heuristic:
  data_heuristic = pm.sample(1000, target_accept=0.95, random_seed=rng)

Let's compare the inferred object permanence capability of these agents, by plotting their capabilities as forest plots:

In [ ]:
forest_plot_random = az.plot_forest(data=data_random['posterior'][['objPermAbility']])
axes_random = forest_plot_random.ravel()[0]
axes_random.set_xlim(left=abilityMin['objPermAbility'], right=abilityMax['objPermAbility'])

forest_plot_heuristic = az.plot_forest(data=data_heuristic['posterior'][['objPermAbility']])
axes_heuristic = forest_plot_heuristic.ravel()[0]
axes_heuristic.set_xlim(left=abilityMin['objPermAbility'], right=abilityMax['objPermAbility'])

We can also look at the summary statistics from the two measurement layouts:

In [ ]:
summary_random = az.summary(data_random['posterior']['objPermAbility'])
summary_random

In [ ]:
summary_heuristic = az.summary(data_heuristic['posterior']['objPermAbility'])
summary_heuristic

We are inferring that the heuristic agent has higher object permanence than the random agent, but both agents are relatively low on object permanence.

## Introducing Navigation

These tasks are fundamentally search tasks. The agent must navigate towards the reward. As such, an agent with object permanence but poor at navigation may fail many of these tasks. Moreover, an agent without object permanence, but that is good at navigating, may accidentally obtain the reward on occasion.

The simplest way to frame this is to say that navigation and object permanence are non-compensatory - being good at navigation does not compensate for being bad at object permanence, and vice versa. For the purposes of this tutorial, we can proceed with this formulation, although it may be more accurate to implement an asymmetric compensatory relationship between these too (since, arguably, navigation is more compensatory for object permanence than vice versa).

Navigation demands can be implemented in terms of how far away the goal is along with the circuitousness of the route. We can define this as the product of distance and number of turns.

Let's extend the measurement layout to include navigation. Let's again define the combined object permanence demand as the distance to the goal (`minDistToGoal`) multiplied by the number of choice (`numChoices`) multiplied by whether or not a goal is present (`goalOccluded`). Let's define navigation as the distance to the goal (`minDistToGoal`) multiplied by the number of turns it takes to get there, avoiding obstacles (`minNumTurnsGoal`).

In [ ]:
def setupOPNavModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility =
  minNavAbility =

  maxPermAbility =
  maxNavAbility =

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility


  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility =

    navAbility =

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistance", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)

    # Margins

    objPermMargin =
    objPermP =

    navP =

    # Define final margin with non-compensatory interaction

    finalP =

    taskSuccess = pm.Bernoulli("Success", finalP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
# @title  A Recommended Solution
def setupOPNavModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).min())
  minNavAbility = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).min())

  maxPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).max())
  maxNavAbility = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).max())

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility


  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)

    navAbility = pm.Uniform("navAbility", minNavAbility, maxNavAbility)

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistance", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)

    # Margins

    objPermMargin = (objPermAbility - ((goalDist* opTest) * numChoices ))
    objPermP = pm.Deterministic("objPermP", logistic999(objPermMargin, min = minPermAbility, max = maxPermAbility))

    navP = pm.Deterministic("navP", logistic999(navAbility - (goalDist * numTurnsGoal), min = minNavAbility, max = maxNavAbility))

    # Define final margin with non-compensatory interaction

    finalP = pm.Deterministic("finalP", (objPermP * navP))

    taskSuccess = pm.Bernoulli("Success", finalP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = setupOPNavModel(opiaagets_dataset, 'Random_Agent_Success')
gv = pm.model_graph.model_to_graphviz(m)
gv

Try this measurement layout on an agent of your choice:

In [ ]:
model, abilityMin, abilityMax = setupOPNavModel(opiaagets_dataset, 'Random_Agent_Success')
with model:
  idata = pm.sample(1000, target_accept=0.95, random_seed=rng)

In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['objPermAbility']])
axes = forest_plot()[0]
axes.set_xlim(left=abilityMin['objPermAbility'], right=abilityMax['objPermAbility'])

In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['navAbility']])
axes = forest_plot.ravel()[0]
axes.set_xlim(left=abilityMin['navAbility'], right=abilityMax['navAbility'])

The cognitive profile for the both agents are as we expect. The heuristic agent is very good at navigating, it follows a rule to go towards any green goals and away from any red lava. The random agent is less good at navigating.


Another feature of the benchmark that ought to affect behaviour is the presence of lava. Let's include that.



First, we have a new metafeature for lava, let's incorporate that into the measurement layout. To do so, we will make use of the Beta prior because the `lavaPresence` is a binary metafeature. We also need to make use of a special margin for these capabilities. Here, if the binary feature is 0, then the margin represents p(success) = 1. If the binary feature is 1, then p(success) = ability.

In [ ]:
def SimplePrMargin(ability, binaryFeature): # must return a value between 0 and 1
  return 1-((1-ability)*binaryFeature)  # If binaryFeature is 0 then the margin represents p(success)=1. If binaryFeature = 1 then p(success)=ability

In [ ]:
def setupOPNavLavaModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility =
  minNavAbility =
  minVAcuityAbility =

  maxPermAbility =
  maxNavAbility =
  maxVAcuityAbility =

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility

  abilityMin["lavaAbility"] = 0
  abilityMax["lavaAbility"] = 1

  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility =

    navAbility =

    lavaAbility =                         # The Beta prior is appropriate for binary variables like this one.

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistance", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)
    lavaPresence = pm.MutableData("lavaPresence", data["lavaPresence"].values)

    # Margins

    objPermMargin =
    objPermP =

    navP =


    ## For this binary feature, we can use the SimplePrMargin function

    lavaP =


    # Define final margin with non-compensatory interaction

    finalP =

    taskSuccess = pm.Bernoulli("ObservedPerformance", finalP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
# @title A Recommended Solution
def setupOPNavLavaModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).min())
  minNavAbility = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).min())

  maxPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).max())
  maxNavAbility = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).max())

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility

  abilityMin["lavaAbility"] = 0
  abilityMax["lavaAbility"] = 1

  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)

    navAbility = pm.Uniform("navAbility", minNavAbility, maxNavAbility)

    lavaAbility = pm.Beta("lavaAbility", 1, 1)

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistance", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)
    lavaPresence = pm.MutableData("lavaPresence", data["lavaPresence"].values)

    # Margins

    objPermMargin = (objPermAbility - ((goalDist * opTest) * numChoices ))
    objPermP = pm.Deterministic("objPermP", logistic999(objPermMargin, min = minPermAbility, max = maxPermAbility))

    navP = pm.Deterministic("navP", logistic999((navAbility - (goalDist * numTurnsGoal)), min = minNavAbility, max = maxNavAbility))

    lavaP = pm.Deterministic("lavaP", SimplePrMargin(lavaAbility, lavaPresence))


    # Define final margin with non-compensatory interaction

    finalP = pm.Deterministic("finalP", (objPermP * navP * lavaP))

    taskSuccess = pm.Bernoulli("Success", finalP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = setupOPNavLavaModel(opiaagets_dataset, 'Random_Agent_Success')
gv = pm.model_graph.model_to_graphviz(m)
gv

Let's see what it estimates for one of the agents:

In [ ]:
model, abilityMin, abilityMax = setupOPNavLavaModel(opiaagets_dataset, 'PPO_Success')
with model:
  idata = pm.sample(1000, target_accept=0.95, random_seed=rng)

In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['objPermAbility']])
axes = forest_plot.ravel()[0]
axes.set_xlim(left=abilityMin['objPermAbility'], right=abilityMax['objPermAbility'])

In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['navAbility']])
axes = forest_plot_random.ravel()[0]
axes.set_xlim(left=abilityMin['navAbility'], right=abilityMax['navAbility'])

These tasks are heavily affected by what the agent can and cannot see. Depending on the input, agents can struggle to observe smaller, more distant goals. So let's introduce a visual acuity ability.

## Introducing Visual Acuity

As in the previous tutorial, we define visual acuity in terms of the size and distance of the goal (when it is visible) from the agent at the start of the episode. We can use `mainGoalSize` for the size metafeature and `euclideanDistanceGoal` for the distance metafeature (since we don't care if there are obstacles in the way, like we do with navigation).

In [ ]:
def setupOPNavLavaVisualModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility =
  minNavAbility =
  minVAcuityAbility =

  maxPermAbility =
  maxNavAbility =
  maxVAcuityAbility =

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility

  abilityMin["lavaAbility"] = 0
  abilityMax["lavaAbility"] = 1

  abilityMin["visualAcuityAbility"] = minVAcuityAbility
  abilityMax["visualAcuityAbility"] = maxVAcuityAbility

  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility =

    navAbility =


    vAcuityAbility =

    # Define environment variables as MutableData

    goalDistNavigation = pm.MutableData("goalDistanceNavigation", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)
    lavaPresence = pm.MutableData("lavaPresence", data["lavaPresence"].values)
    goalSize = pm.MutableData("goalSize", data["mainGoalSize"].values)
    goalDistanceVisual  = pm.MutableData("goalDistanceVisual", data["euclideanDistanceGoal"].values)

    # Margins

    objPermMargin =
    objPermP =

    navP =

    lavaP =

    visualAcuityP =

    # Define final margin with non-compensatory interaction

    finalP =

    taskSuccess = pm.Bernoulli("Success", finalP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
# @title A Recommended Solution
def setupOPNavLavaVisualModel(data, agent_col_name: str):

  # get results column
  results = data[agent_col_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).min())
  minNavAbility = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).min())
  minVAcuityAbility = ((data["euclideanDistanceGoal"]/data["mainGoalSize"]).min())

  maxPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).max())
  maxNavAbility = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).max())
  maxVAcuityAbility = ((data["euclideanDistanceGoal"]/data["mainGoalSize"]).max())

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility

  abilityMin["lavaAbility"] = 0
  abilityMax["lavaAbility"] = 1

  abilityMin["visualAcuityAbility"] = minVAcuityAbility
  abilityMax["visualAcuityAbility"] = maxVAcuityAbility

  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)

    navAbility = pm.Uniform("navAbility", minNavAbility, maxNavAbility)

    lavaAbility = pm.Beta("lavaAbility", 1, 1)

    vAcuityAbility = pm.Uniform("visualAcuityAbility", minVAcuityAbility, maxVAcuityAbility)

    # Define environment variables as MutableData

    goalDistNavigation = pm.MutableData("goalDistanceNavigation", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)
    lavaPresence = pm.MutableData("lavaPresence", data["lavaPresence"].values)
    goalSize = pm.MutableData("goalSize", data["mainGoalSize"].values)
    goalDistanceVisual  = pm.MutableData("goalDistanceVisual", data["euclideanDistanceGoal"].values)

    # Margins

    objPermMargin = (objPermAbility - ((goalDistNavigation* opTest) * numChoices * opTest))
    objPermP = pm.Deterministic("objPermP", logistic999(objPermMargin, min = minPermAbility, max = maxPermAbility))

    navP = pm.Deterministic("navP", logistic999((navAbility - (goalDistNavigation * numTurnsGoal)), min = minNavAbility, max = maxNavAbility))

    lavaP = pm.Deterministic("lavaP", SimplePrMargin(lavaAbility, lavaPresence))

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic999((np.log(vAcuityAbility) - np.log(goalDistanceVisual/goalSize)), min = minVAcuityAbility, max = maxVAcuityAbility))

    # Define final margin with non-compensatory interaction

    finalP = pm.Deterministic("finalP", (objPermP * navP * lavaP * visualAcuityP))

    taskSuccess = pm.Bernoulli("Success", finalP, observed = results)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = setupOPNavLavaVisualModel(opiaagets_dataset, 'Random_Agent_Success')
gv = pm.model_graph.model_to_graphviz(m)
gv

Play around with the estimates for the different agents:

In [ ]:
model, abilityMin, abilityMax = setupOPNavLavaVisualModel(opiaagets_dataset, 'Dreamer_Success')
with model:
  idata = pm.sample(1000, tune = 500, target_accept=0.95, random_seed=rng)


In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['objPermAbility']])
axes = forest_plot.ravel()[0]
axes.set_xlim(left=abilityMin['objPermAbility'], right=abilityMax['objPermAbility'])


In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['navAbility']])
axes = forest_plot.ravel()[0]
axes.set_xlim(left=abilityMin['navAbility'], right=abilityMax['navAbility'])

In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['lavaAbility']])
axes = forest_plot.ravel()[0]
axes.set_xlim(left=abilityMin['lavaAbility'], right=abilityMax['lavaAbility'])

# Extending To The Multivariate Case

The test battery has been extended to incorporate another kind of object permanence test, as well as another response variable: whether the agent chose the correct choice or not. In tasks where there was no choice to be made, the correct choice is simply whether they succeeded on the task.

Now let's introduce the second response variable, `correctChoice`, and the corresponding metafeatures. There is the navigation demand of navigating to the goal, which is quite high in both the PCTB and CV Chick tasks. However, there is also the navigation demand of navigating to the choice point, which is high for the PCTB tasks (equivalent to navigating to the goal), but low for the CV Chick tasks (they only need to navigate to the end of the ramp). Find a possible implementation below:

Feel free to play around with how everything is connected:

In [ ]:
def setupMultivariateModel(data, agent_success_name: str, agent_choice_name: str):

  # get results column
  successes = data[agent_success_name]
  choices = data[agent_choice_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minSuccessNav = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).min())
  minChoiceNav = ((data["navigationDistanceChoice"] * data["navigationTurnsChoice"]).min())

  maxSuccessNav = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).max())
  maxChoiceNav = ((data["navigationDistanceChoice"] * data["navigationTurnsChoice"]).max())

  minPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).min())
  minNavAbility = min([minSuccessNav, minChoiceNav])
  minVAcuityAbility = ((data["euclideanDistanceGoal"]/data["mainGoalSize"]).min())

  maxPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).max())
  maxNavAbility = max([maxSuccessNav, maxChoiceNav])
  maxVAcuityAbility = ((data["euclideanDistanceGoal"]/data["mainGoalSize"]).max())

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility

  abilityMin["visualAcuityAbility"] = minVAcuityAbility
  abilityMax["visualAcuityAbility"] = maxVAcuityAbility

  abilityMin["lavaAbility"] = 0
  abilityMax["lavaAbility"] = 1

  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)

    navAbility = pm.Uniform("navAbility", minNavAbility, maxNavAbility)

    vAcuityAbility = pm.Uniform("visualAcuityAbility", minVAcuityAbility, maxVAcuityAbility)

    lavaAbility = pm.Beta("lavaAbility", 1, 1)

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistanceNavigation", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)
    goalSize = pm.MutableData("goalSize", data["mainGoalSize"].values)
    lavaPresence = pm.MutableData("lavaPresence", data["lavaPresence"].values)
    goalDistanceVisual  = pm.MutableData("goalDistanceVisual", data["euclideanDistanceGoal"].values)

    choiceDist = pm.MutableData("goalDistanceChoice", data["navigationDistanceGoal"].values)
    numTurnsChoice = pm.MutableData("navigationTurnsChoice", data["navigationTurnsChoice"].values)

    # Margins

    objPermMarginSuccess = (objPermAbility - ((goalDist * opTest) * numChoices))
    objPermSuccessP = pm.Deterministic("objPermSuccessP", logistic999(objPermMarginSuccess, min = minPermAbility, max = maxPermAbility))

    objPermMarginChoice = (objPermAbility - ((choiceDist * opTest) * numChoices ))
    objPermChoiceP = pm.Deterministic("objPermChoiceP", logistic999(objPermMarginChoice, min = minPermAbility, max = maxPermAbility))

    lavaP = pm.Deterministic("lavaP", logistic999(lavaAbility - lavaPresence, min = 0, max = 1))

    navSuccessP = pm.Deterministic("navSuccessP", logistic999((navAbility - (goalDist * numTurnsGoal)), min = minNavAbility, max = maxNavAbility) * lavaP)
    navChoiceP = pm.Deterministic("navChoiceP", logistic999((navAbility - (choiceDist * numTurnsChoice)), min = minNavAbility, max = maxNavAbility))

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic999((np.log(vAcuityAbility) - np.log(goalDistanceVisual/goalSize)), min = minVAcuityAbility, max = maxVAcuityAbility))

    # Define final margin with non-compensatory interaction

    choiceP = pm.Deterministic("choiceP", (navChoiceP * objPermChoiceP * visualAcuityP))
    successP = pm.Deterministic("successP", (navSuccessP * objPermSuccessP))

    taskSuccess = pm.Bernoulli("Success", successP, observed=successes)
    taskChoice = pm.Bernoulli("CorrectChoice", choiceP, observed=choices)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = setupMultivariateModel(opiaagets_dataset, 'Random_Agent_Success', 'Random_Agent_Choice')
gv = pm.model_graph.model_to_graphviz(m)
gv

Finally, here's the most complex measurement layout, with the most capabilities, that we can build with the data available. Have a play around with the different connections:

In [ ]:
def setupComplexMultivariateModel(data, agent_success_name: str, agent_choice_name: str):

  # get results column
  successes = data[agent_success_name]
  choices = data[agent_choice_name]

  # define bounds
  abilityMin = {}
  abilityMax = {}

  minSuccessNav = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).min())
  minChoiceNav = ((data["navigationDistanceChoice"] * data["navigationTurnsChoice"]).min())

  maxSuccessNav = ((data["navigationDistanceGoal"] * data["navigationTurnsGoal"]).max())
  maxChoiceNav = ((data["navigationDistanceChoice"] * data["navigationTurnsChoice"]).max())

  minPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).min())
  minNavAbility = min([minSuccessNav, minChoiceNav])
  minVAcuityAbility = ((data["navigationDistanceGoal"]/data["mainGoalSize"]).min())

  maxPermAbility = ((data["navigationDistanceGoal"] * data["numChoices"]).max())
  maxNavAbility = max([maxSuccessNav, maxChoiceNav])
  maxVAcuityAbility = ((data["navigationDistanceGoal"]/data["mainGoalSize"]).max())

  abilityMin["objPermAbility"] = minPermAbility
  abilityMax["objPermAbility"] = maxPermAbility

  abilityMin["navAbility"] = minNavAbility
  abilityMax["navAbility"] = maxNavAbility

  abilityMin["visualAcuityAbility"] = minVAcuityAbility
  abilityMax["visualAcuityAbility"] = maxVAcuityAbility

  abilityMin["lavaAbility"] = 0
  abilityMax["lavaAbility"] = 1

  abilityMin["rightAbility"] = 0
  abilityMax["rightAbility"] = 1

  abilityMin["leftAbility"] = 0
  abilityMax["leftAbility"] = 1

  abilityMin["aheadAbility"] = 0
  abilityMax["aheadAbility"] = 1

  m = pm.Model()
  with m:

    # Define abilities and their priors

    objPermAbility = pm.Uniform("objPermAbility", minPermAbility, maxPermAbility)

    navAbility = pm.Uniform("navAbility", minNavAbility, maxNavAbility)

    vAcuityAbility = pm.Uniform("visualAcuityAbility", minVAcuityAbility, maxVAcuityAbility)

    lavaAbility = pm.Beta("lavaAbility", 1, 1)

    rightAbility = pm.Beta("rightAbility", 1, 1)

    centreAbility = pm.Beta("centreAbility", 1, 1)

    leftAbility = pm.Beta("leftAbility", 1, 1)

    # Define environment variables as MutableData

    goalDist = pm.MutableData("goalDistanceNavigation", data["navigationDistanceGoal"].values)
    numChoices = pm.MutableData("numChoices", data["numChoices"].values)
    opTest = pm.MutableData("goalOccluded", data["goalOccluded"].values)
    numTurnsGoal = pm.MutableData("navigationTurnsGoal", data["navigationTurnsGoal"].values)
    goalSize = pm.MutableData("goalSize", data["mainGoalSize"].values)
    lavaPresence = pm.MutableData("lavaPresence", data["lavaPresence"].values)
    #goalDistanceVisual  = pm.MutableData("goalDistanceVisual", data["euclideanDistanceGoal"].values)

    choiceDist = pm.MutableData("goalDistanceChoice", data["navigationDistanceGoal"].values)
    numTurnsChoice = pm.MutableData("navigationTurnsChoice", data["navigationTurnsChoice"].values)

    goalRight = pm.MutableData("goalRight", data["goalRight"].values)
    goalCentre = pm.MutableData("goalCentre", data["goalCentre"].values)
    goalLeft = pm.MutableData("goalLeft", data["goalLeft"].values)

    # Margins

    objPermMarginSuccess = (objPermAbility - ((goalDist * opTest) * numChoices))
    objPermSuccessP = pm.Deterministic("objPermSuccessP", logistic999(objPermMarginSuccess, min = minPermAbility, max = maxPermAbility))

    objPermMarginChoice = (objPermAbility - ((choiceDist * opTest) * numChoices ))
    objPermChoiceP = pm.Deterministic("objPermChoiceP", logistic999(objPermMarginChoice, min = minPermAbility, max = maxPermAbility))

    lavaP = pm.Deterministic("lavaP", SimplePrMargin(lavaAbility, lavaPresence))

    rightP = pm.Deterministic("rightP", SimplePrMargin(rightAbility, goalRight))
    centreP = pm.Deterministic("centreP", SimplePrMargin(centreAbility, goalCentre))
    leftP = pm.Deterministic("leftP", SimplePrMargin(leftAbility, goalLeft))

    navSuccessP = pm.Deterministic("navSuccessP", logistic999((navAbility - (goalDist * numTurnsGoal)), min = minNavAbility, max = maxNavAbility) * lavaP)
    navChoiceP = pm.Deterministic("navChoiceP", logistic999((navAbility - (choiceDist * numTurnsChoice)), min = minNavAbility, max = maxNavAbility) * rightP * centreP * leftP)

    visualAcuityP = pm.Deterministic("visualAcuityP", logistic999((np.log(vAcuityAbility) - np.log(goalDist/goalSize)), min = minVAcuityAbility, max = maxVAcuityAbility))

    # Define final margin with non-compensatory interaction

    choiceP = pm.Deterministic("choiceP", (navChoiceP * objPermChoiceP ))
    successP = pm.Deterministic("successP", (navSuccessP * objPermSuccessP * visualAcuityP))

    taskSuccess = pm.Bernoulli("Success", successP, observed=successes)
    taskChoice = pm.Bernoulli("CorrectChoice", choiceP, observed=choices)

  return m, abilityMin, abilityMax

In [ ]:
m, abilityMin, abilityMax = setupComplexMultivariateModel(opiaagets_dataset, 'Random_Agent_Success', 'Random_Agent_Choice')
gv = pm.model_graph.model_to_graphviz(m)
gv

Try fitting it to an agent:

In [ ]:
model, abilityMin, abilityMax = setupComplexMultivariateModel(human_dataset, 'Human_Success', 'Human_Choice')
with model:
  idata = pm.sample(1000, tune = 500, target_accept=0.95, random_seed=rng)


And plotting some of its capabilities:

In [ ]:
forest_plot = az.plot_forest(data=idata['posterior'][['objPermAbility']])
axes = forest_plot.ravel()[0]
axes.set_xlim(left=abilityMin['objPermAbility'], right=abilityMax['objPermAbility'])

# BONUS

If there is time at the end:

1. Try running some diagnostics on the fitted models:

In [ ]:
az.plot_trace(idata['posterior'][['objPermAbility']])
az.plot_energy(idata)
az.summary(idata['posterior'][['objPermAbility', 'navAbility', 'visualAcuityAbility']])

2. Play around with the models to try to improve their fit (they're works-in-progress!) and predictiveness.
3. Build a large multi-level measurement layout for running inferences on multiple agents simultaneously. A place to start for implementing this would be [here](https://www.pymc.io/projects/examples/en/latest/generalized_linear_models/multilevel_modeling.html).